# Using Qdrant as Vector Database

In [1]:
from utils import (Path, PromptTemplate, PyPDFLoader, StrOutputParser,
                   itemgetter, load_embedding_model, load_model,
                   qdrant_vectorstore)


## Content

1. [Creating a vector database with Qdrant](#1.-Creating-a-vector-database-with-Qdrant)
2. [Information Retrieval (Search)](#2.-Information-Retrieval)
3. [Use an LLM to ask to the retrieved information](#3.-Use-an-LLM-to-ask-to-the-retrieved-information)



##  1. Creating a vector database with Qdrant

**Loading Documents**

In [2]:
file_path = Path(".") / "database" / "pdfs" / "doc2.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
pages

[Document(page_content='CONGRESO DE LOS DIPUTADOS\nXV LEGISLATURA\nSerie B:  \nPROPOSICIONES DE LEY  15 de marzo de 2024  Núm. 75-1  Pág. 1BOLETÍN OFICIAL  \nDE LAS CORTES GENERALES\nPROPOSICIÓN DE LEY\n122/000064 Proposición de Ley por la que se modifica la Ley 12/2013, \nde 2 de agosto, de medidas para mejorar el funcionamiento \nde la cadena alimentaria.\nPresentada por el Grupo Parlamentario Republicano\nLa Mesa de la Cámara, en su reunión del día de hoy, ha adoptado el acuerdo que se \nindica respecto del asunto de referencia.\n(122) Proposición de ley de Grupos Parlamentarios del Congreso.\nAutor: Grupo Parlamentario Republicano\nProposición de Ley por la que se modifica la Ley 12/2013, de 2 de agosto, de medidas \npara mejorar el funcionamiento de la cadena alimentaria.\nAcuerdo:\nTeniendo en cuenta los antecedentes incorporados mediante escrito número de \nregistro 15391, admitir a trámite, trasladar al Gobierno a los efectos del artículo 126 del \nReglamento, publicar en el Bo

**Convert Documents to Vectors and Push to Qdrant**

In [3]:
MODEL_EMBEDDINGS = "text-embeddings-3-small"
embeddings = load_embedding_model(source="openai", name=None)

qdrant = qdrant_vectorstore(documents=pages, embeddings=embeddings)

## 2. Information Retrieval

### Search options

- Similarity search: 
- Similarity search with score
- Metadata filtering
- Maximum marginal relevance search (MMR)

**Similarity search**:

In [4]:
query = "Cuál es el artículo 4 de la ley propuesta?"
found_docs = qdrant.similarity_search(query)
print(found_docs)
print(found_docs[0].metadata)

[Document(page_content='CONGRESO DE LOS DIPUTADOS\nXV LEGISLATURA\nSerie B:  \nPROPOSICIONES DE LEY  15 de marzo de 2024  Núm. 75-1  Pág. 1BOLETÍN OFICIAL  \nDE LAS CORTES GENERALES\nPROPOSICIÓN DE LEY\n122/000064 Proposición de Ley por la que se modifica la Ley 12/2013, \nde 2 de agosto, de medidas para mejorar el funcionamiento \nde la cadena alimentaria.\nPresentada por el Grupo Parlamentario Republicano\nLa Mesa de la Cámara, en su reunión del día de hoy, ha adoptado el acuerdo que se \nindica respecto del asunto de referencia.\n(122) Proposición de ley de Grupos Parlamentarios del Congreso.\nAutor: Grupo Parlamentario Republicano\nProposición de Ley por la que se modifica la Ley 12/2013, de 2 de agosto, de medidas \npara mejorar el funcionamiento de la cadena alimentaria.\nAcuerdo:\nTeniendo en cuenta los antecedentes incorporados mediante escrito número de \nregistro 15391, admitir a trámite, trasladar al Gobierno a los efectos del artículo 126 del \nReglamento, publicar en el Bo

**Similarity search with score**:

In [5]:
found_docs = qdrant.similarity_search_with_score(query)
for doc, score in found_docs:
    print(f"Page: {doc.metadata['page']}, Score: {score}, Content: {doc.page_content[:50]}...")

Page: 0, Score: 0.8149556, Content: CONGRESO DE LOS DIPUTADOS
XV LEGISLATURA
Serie B: ...
Page: 0, Score: 0.8149556, Content: CONGRESO DE LOS DIPUTADOS
XV LEGISLATURA
Serie B: ...
Page: 0, Score: 0.8149556, Content: CONGRESO DE LOS DIPUTADOS
XV LEGISLATURA
Serie B: ...
Page: 3, Score: 0.8125105, Content: BOLETÍN OFICIAL DE LAS CORTES GENERALES
CONGRESO D...


**Metadata filtering:**

In [6]:
from qdrant_client.http import models as rest



# Define the filter
page_filter = rest.Filter(
    must=[
        rest.FieldCondition(
            key="metadata.page",
            match=rest.MatchValue(value=3)
        )
    ]
)
found_docs = qdrant.similarity_search_with_score(
    query,
    filter=page_filter
)

for doc, score in found_docs:
    print(f"Page: {doc.metadata['page']}, Score: {score}, Content: {doc.page_content[:50]}...")

Page: 3, Score: 0.8125105, Content: BOLETÍN OFICIAL DE LAS CORTES GENERALES
CONGRESO D...
Page: 3, Score: 0.8125105, Content: BOLETÍN OFICIAL DE LAS CORTES GENERALES
CONGRESO D...
Page: 3, Score: 0.8125105, Content: BOLETÍN OFICIAL DE LAS CORTES GENERALES
CONGRESO D...
Page: 3, Score: 0.77967703, Content: que queda redactado como sigue:
«c) Precio del con...


**Maximum marginal relevance search (MMR):**

In [7]:
found_docs = qdrant.max_marginal_relevance_search(query, k=2, fetch_k=10)
found_docs

[Document(page_content='CONGRESO DE LOS DIPUTADOS\nXV LEGISLATURA\nSerie B:  \nPROPOSICIONES DE LEY  15 de marzo de 2024  Núm. 75-1  Pág. 1BOLETÍN OFICIAL  \nDE LAS CORTES GENERALES\nPROPOSICIÓN DE LEY\n122/000064 Proposición de Ley por la que se modifica la Ley 12/2013, \nde 2 de agosto, de medidas para mejorar el funcionamiento \nde la cadena alimentaria.\nPresentada por el Grupo Parlamentario Republicano\nLa Mesa de la Cámara, en su reunión del día de hoy, ha adoptado el acuerdo que se \nindica respecto del asunto de referencia.\n(122) Proposición de ley de Grupos Parlamentarios del Congreso.\nAutor: Grupo Parlamentario Republicano\nProposición de Ley por la que se modifica la Ley 12/2013, de 2 de agosto, de medidas \npara mejorar el funcionamiento de la cadena alimentaria.\nAcuerdo:\nTeniendo en cuenta los antecedentes incorporados mediante escrito número de \nregistro 15391, admitir a trámite, trasladar al Gobierno a los efectos del artículo 126 del \nReglamento, publicar en el Bo

**Qdrant as retriever:**

In [8]:
retriever = qdrant.as_retriever(
    search_type="mmr", # “similarity” (default), “mmr”, or “similarity_score_threshold”
    search_kwargs=dict(
        k=4,
        score_threshold=0.5,
        fetch_k=10,
        lambda_mult=0.5,
        # filter=page_filter
    )
)
retriever.invoke(query)

[Document(page_content='CONGRESO DE LOS DIPUTADOS\nXV LEGISLATURA\nSerie B:  \nPROPOSICIONES DE LEY  15 de marzo de 2024  Núm. 75-1  Pág. 1BOLETÍN OFICIAL  \nDE LAS CORTES GENERALES\nPROPOSICIÓN DE LEY\n122/000064 Proposición de Ley por la que se modifica la Ley 12/2013, \nde 2 de agosto, de medidas para mejorar el funcionamiento \nde la cadena alimentaria.\nPresentada por el Grupo Parlamentario Republicano\nLa Mesa de la Cámara, en su reunión del día de hoy, ha adoptado el acuerdo que se \nindica respecto del asunto de referencia.\n(122) Proposición de ley de Grupos Parlamentarios del Congreso.\nAutor: Grupo Parlamentario Republicano\nProposición de Ley por la que se modifica la Ley 12/2013, de 2 de agosto, de medidas \npara mejorar el funcionamiento de la cadena alimentaria.\nAcuerdo:\nTeniendo en cuenta los antecedentes incorporados mediante escrito número de \nregistro 15391, admitir a trámite, trasladar al Gobierno a los efectos del artículo 126 del \nReglamento, publicar en el Bo

## 3. Use an LLM to ask to the retrieved information

In [11]:
MODEL = "gpt-3.5-turbo"

model = load_model(MODEL)
parser = StrOutputParser()

template = """
Responde a la siguiente pregunta basándote en el contexto a continuación. Si no puedes responder, escribe "No sé".

Contexto: {context}

Pregunta: {question}
"""

prompt = PromptTemplate.from_template(template)
retriever = qdrant.as_retriever()

In [12]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [13]:
questions = [
    "Detalla el título, la fecha, el autor y todos los stakeholders de esta proposición de ley.",
    "De forma muy resumida, ¿por qué es importante esta ley?",
    "De forma muy resumida, ¿a quién afecta esta ley?",
    "Lista y describe los puntos clave de esta ley."
]

for q in questions:
    print(f"Question: {q}")
    print(f"Answer: {chain.invoke({'question': q})}")
    print()

Question: Detalla el título, la fecha, el autor y todos los stakeholders de esta proposición de ley.
Answer: Título: Proposición de Ley por la que se modifica la Ley 12/2013, de 2 de agosto, de medidas para mejorar el funcionamiento de la cadena alimentaria.
Fecha: 15 de marzo de 2024
Autor: Grupo Parlamentario Republicano
Stakeholders: Grupo Parlamentario Republicano, Mesa de la Cámara, Gobierno, agricultores, sector primario, productores, industria, distribución, Agencia de Información y Control Alimentario, gran distribución, Parlamento Europeo, Consejo, operadores de la cadena, productores primarios.

Question: De forma muy resumida, ¿por qué es importante esta ley?
Answer: Esta ley es importante porque busca modificar la Ley de la Cadena Alimentaria para garantizar precios justos en los productos agrarios y permitir que los agricultores puedan vivir dignamente de su trabajo. Además, busca prohibir la reventa a pérdidas en la cadena alimentaria y establecer normas claras para regul